In [ ]:
PROFILE MATCH (up:Class:Neuron)<-[:SUBCLASSOF|INSTANCEOF*..]-(n1:Neuron:Individual) WHERE up.short_form = "FBbt_00111272" MATCH (n1)-[r:synapsed_to]->(n2:Neuron:Individual) WHERE r.weight[0] >= 5 OPTIONAL MATCH (n2)-[:SUBCLASSOF|INSTANCEOF*..]->(down:Class:Neuron) WHERE down.short_form = "FBbt_00005106" OPTIONAL MATCH (c1:Class)<-[:INSTANCEOF]-(n1), (c2:Class)<-[:INSTANCEOF]-(n2) OPTIONAL MATCH (n1)-[r1:database_cross_reference]->(s1:Site) WHERE exists(s1.is_data_source) AND s1.is_data_source = True OPTIONAL MATCH (n2)-[r2:database_cross_reference]->(s2:Site) WHERE exists(s1.is_data_source) AND s2.is_data_source = True RETURN n1.short_form as upstream_neuron_id, n1.label as upstream_neuron_name, r.weight[0] as weight, n2.short_form as downstream_neuron_id, n2.label as downstream_neuron_name, apoc.text.join(collect(distinct c1.label),'|') AS upstream_class, apoc.text.join(collect(distinct c2.label),'|') as downstream_class, s1.short_form AS up_data_source, r1.accession[0] as up_accession,s2.short_form AS down_source, r2.accession[0] AS down_accession

Query starting point:
	Cell Type:
		In which datasets can I find this cell type (might start from a synonym)
	What is NT for this cell type.
	NTR profiles of starting neuron
	Connectivity query by type → Upstream/Downstream neurons
	What NT do these have?  (Can we break this down by evidence?)
	NTR profiles of downstream neurons corresponding to NT starting neuron 
	NTR profile of starting neuron corresponding to downstream neurons
	
	What motifs are users looking for?  Looking for feedback motifs?  E.g. excitatory → GABA ergic → feedback to starting excitatory neuron.


 What's missing: Discovery query?  Find which neurons we have transcriptomic data for before proceeding?


ModuleNotFoundError: No module named 'vfb_connect'

In [2]:
!pip install vfb_connect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from vfb_connect.cross_server_tools import VfbConnect
vc = VfbConnect()

/Users/do12/Documents/GitHub/VFB_connect/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
#Cell Type:
#		In which datasets can I find this cell type (might start from a synonym)
#	What is NT for this cell type.

# We can't start from a synonym right now.  Need to add a new lookup
# We can look at term info for an individual neuron
vc.neo_query_wrapper.get_type_TermInfo([vc.lookup_id('Dm9')], summary=True) 
# Improvements: Make summary True by default. Support dataframe output by default for summary


[{'label': 'distal medullary amacrine neuron Dm9',
  'symbol': 'Dm9',
  'id': 'FBbt_00111271',
  'tags': 'Entity|Adult|Anatomy|Cell|Class|Glutamatergic|Nervous_system|Neuron|Visual_system|hasScRNAseq',
  'parents_label': 'Notch OFF hemilineage secondary neuron|adult glutamatergic neuron|distal medullary amacrine neuron',
  'parents_id': 'FBbt_00049542|FBbt_00058208|FBbt_00003767'}]

In [6]:
# From this we can see that this type is glutamatergic.  For whey we can look at description (although notes 
#- in some cases the infor might be on a superclass)
vc.neo_query_wrapper.get_type_TermInfo([vc.lookup_id('Dm9')])[0]['term']['description'][0]



'Distal medullary wide-field amacrine neuron with arbors that span multiple columns in layers M1 and M6A and columnar processes between these two layers. The processes in M2 and M5 occupy a central column position and are closely associated with R7 and R8 photoreceptor axons, covering around 7 columns. There are around 110 Dm9 neurons per hemisphere. They are glutamatergic (Davis et al., 2020).'

In [17]:
vc.neo_query_wrapper.get_type_TermInfo([vc.lookup_id('Dm9')])[0]['def_pubs']

[{'core': {'symbol': '',
   'iri': 'http://flybase.org/reports/FBrf0228639',
   'types': ['Entity', 'Individual', 'pub'],
   'short_form': 'FBrf0228639',
   'unique_facets': ['pub'],
   'label': 'Nern et al., 2015, Proc. Natl. Acad. Sci. U.S.A. 112(22): E2967--E2976'},
  'FlyBase': 'FBrf0228639',
  'PubMed': '25964354',
  'DOI': '10.1073/pnas.1506763112'},
 {'core': {'symbol': '',
   'iri': 'http://flybase.org/reports/FBrf0244935',
   'types': ['Entity', 'Individual', 'pub'],
   'short_form': 'FBrf0244935',
   'unique_facets': ['pub'],
   'label': 'Davis et al., 2020, eLife 9: e50901'},
  'FlyBase': 'FBrf0244935',
  'PubMed': '31939737',
  'DOI': '10.7554/eLife.50901'},
 {'core': {'symbol': '',
   'iri': 'http://flybase.org/reports/FBrf0222324',
   'types': ['Entity', 'Individual', 'pub'],
   'short_form': 'FBrf0222324',
   'unique_facets': ['pub'],
   'label': 'Takemura et al., 2013, Nature 500(7461): 175--181'},
  'FlyBase': 'FBrf0222324',
  'PubMed': '23925240',
  'DOI': '10.1038/na

In [ ]:
# Supporting ref = label': 'Davis et al., 2020, eLife 9: e50901'

In [18]:
vc.get_connected_neurons_by_type(upstream_type='Dm9', downstream_type='Dm', weight=1) # Note 

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_001029fa,Dm9#7 (FAFB:11916195),2,VFB_001029fc,Dm2#7 (FAFB:11918130),distal medullary amacrine neuron Dm9,distal medullary amacrine neuron Dm2,None,None,None,None
1,VFB_001029c5,Dm9#2 (FAFB:11447061),4,VFB_001029ai,Dm8#4 (FAFB:10208775),distal medullary amacrine neuron Dm9,distal medullary amacrine neuron Dm8,None,None,None,None
2,VFB_001029cr,Dm9#1 (FAFB:11452427),2,VFB_001029ai,Dm8#4 (FAFB:10208775),distal medullary amacrine neuron Dm9,distal medullary amacrine neuron Dm8,None,None,None,None
3,VFB_001029c5,Dm9#2 (FAFB:11447061),3,VFB_001029ae,Dm8#1 (FAFB:10109586),distal medullary amacrine neuron Dm9,distal medullary amacrine neuron Dm8,None,None,None,None
4,VFB_001029bq,Dm9#9 (FAFB:10657500),2,VFB_001029as,Dm-DRA2#1 (FAFB:10411788),distal medullary amacrine neuron Dm9,distal medullary amacrine neuron Dm-DRA2,None,None,None,None


# Notes - 

* we have a problem with queries running indefinitely when we go above Dm as downstream neuron type.
   * We could deal with this for the most general use-case  - Neuron by making all args optional but failing if at least one of upstream or downstream are filled.  There's no need to query up to neuron because we know this form the Semantic tag
* Need to fix typ on in data source return
   `                        "WHERE exists(s1.is_data_source) AND s2.is_data_source = True " \`
  -->
  `                        "WHERE exists(s2.is_data_source) AND s2.is_data_source = True " \`

With the second fixed we will return datasets - BUT the dataset names are really not easy to understand.  THIS NEEDS TO BE FIXED. There has already been some work on this to add SYMBOLS.  We need to be returning these in queries!

In [ ]:
# 	NTR profiles of downstream neurons corresponding to NT starting neuron 


In [23]:
# Dm9 downstream types
dm9_ds_type= set(vc.get_connected_neurons_by_type(upstream_type='Dm9', downstream_type='Dm', weight=1)['downstream_class'])

In [24]:
dm9_ds_type

{'distal medullary amacrine neuron Dm-DRA2',
 'distal medullary amacrine neuron Dm2',
 'distal medullary amacrine neuron Dm8'}

In [25]:
vc.get_gene_function_filters()

['Acetylcholine_receptor',
 'Calcium_binding',
 'Carbohydrate_binding',
 'Chaperone',
 'Cytoskeleton_binding',
 'DNA_binding',
 'Dopamine_receptor',
 'Enzyme',
 'GABA_receptor',
 'GPCR',
 'Glutamate_receptor',
 'Gustatory_receptor',
 'Histamine_receptor',
 'Hormone',
 'Ion_channel',
 'Ligand',
 'Lipid_binding',
 'Mechanosensory_ion_channel',
 'Metal_ion_binding',
 'Neuropeptide',
 'Neurotransmitter_receptor',
 'Octopamine_receptor',
 'Odorant_binding',
 'Olfactory_receptor',
 'Peptide_or_protein_hormone_receptor',
 'Photoreceptor',
 'RNA_binding',
 'Receptor',
 'Regulator',
 'Serotonin_receptor',
 'Small_molecule_binding',
 'Structural_molecule',
 'Thermosensory_ion_channel',
 'Transcription_factor',
 'Transporter',
 'Tyramine_receptor']

In [26]:
vc.get_transcriptomic_profile?


Signature:
vc.get_transcriptomic_profile(
    cell_type,
    gene_type=False,
    return_dataframe=True,
)
Docstring:
Get gene expression data for a given cell_type.

Returns a DataFrame of gene expression data for clusters of cells annotated as cell_type (or subtypes).
Can optionally restrict to a gene_type - these can be retrieved by running get_gene_function_filters.
If no data is found, returns False.

:param cell_type: The ID, name or symbol of a class in the Drosophila Anatomy Ontology (FBbt).
:param gene_type: Optional. A gene function label - these can be retrieved by running get_gene_function_filters().
:return: DataFrame with gene expression data for clusters of cells annotated as cell_type (or subtypes).
:rtype: DataFrame
File:      ~/Documents/GitHub/VFB_connect/venv/lib/python3.9/site-packages/vfb_connect/cross_server_tools.py
Type:      method

In [30]:
for t in dm9_ds_type:
    print("Getting transcriptomic profile for " + t)
    print(vc.get_transcriptomic_profile(t, gene_type='Glutamate_receptor'))
    

Getting transcriptomic profile for distal medullary amacrine neuron Dm-DRA2
Empty DataFrame
Columns: []
Index: []
Getting transcriptomic profile for distal medullary amacrine neuron Dm2
                              cell_type   cell_type_id sample_sex  \
0  distal medullary amacrine neuron Dm2  FBbt_00003769       None   
1  distal medullary amacrine neuron Dm2  FBbt_00003769       None   

        sample_tissue                                          ref    gene  \
0  [adult optic lobe]  Özel et al., 2021, Nature 589(7840): 88--95  GluClα   
1  [adult optic lobe]  Özel et al., 2021, Nature 589(7840): 88--95  GluRIB   

       gene_id                                           function    extent  \
0  FBgn0024963  [Receptor, Neurotransmitter_receptor, Ion_chan...  0.451883   
1  FBgn0264000  [Receptor, Neurotransmitter_receptor, Ion_chan...  0.305439   

       level  
0  1098.2018  
1  1094.0442  
Getting transcriptomic profile for distal medullary amacrine neuron Dm8
                

In [31]:
vc.get_transcriptomic_profile('Dm2', gene_type='Glutamate_receptor')

,cell_type,cell_type_id,sample_sex,sample_tissue,ref,gene,gene_id,function,extent,level
0,distal medullary amacrine neuron Dm2,FBbt_00003769,None,[adult optic lobe],"Özel et al., 2021, Nature 589(7840): 88--95",GluClα,FBgn0024963,"[Receptor, Neurotransmitter_receptor, Ion_chan...",0.451883,1098.2018
1,distal medullary amacrine neuron Dm2,FBbt_00003769,None,[adult optic lobe],"Özel et al., 2021, Nature 589(7840): 88--95",GluRIB,FBgn0264000,"[Receptor, Neurotransmitter_receptor, Ion_chan...",0.305439,1094.0442


In [32]:
vc.get_connected_neurons_by_type(upstream_type='Dm', downstream_type='Dm', weight=1)

,upstream_neuron_id,upstream_neuron_name,weight,downstream_neuron_id,downstream_neuron_name,upstream_class,downstream_class,up_data_source,up_accession,down_source,down_accession
0,VFB_001029ai,Dm8#4 (FAFB:10208775),22,VFB_001029cr,Dm9#1 (FAFB:11452427),distal medullary amacrine neuron Dm8,distal medullary amacrine neuron Dm9,None,None,None,None
1,VFB_001029ae,Dm8#1 (FAFB:10109586),23,VFB_001029cr,Dm9#1 (FAFB:11452427),distal medullary amacrine neuron Dm8,distal medullary amacrine neuron Dm9,None,None,None,None
2,VFB_001029ae,Dm8#1 (FAFB:10109586),7,VFB_001029ee,Dm9#4 (FAFB:11484679),distal medullary amacrine neuron Dm8,distal medullary amacrine neuron Dm9,None,None,None,None
3,VFB_001029ah,Dm8#11 (FAFB:10196189),8,VFB_001029ee,Dm9#4 (FAFB:11484679),distal medullary amacrine neuron Dm8,distal medullary amacrine neuron Dm9,None,None,None,None
4,VFB_001029ah,Dm8#11 (FAFB:10196189),3,VFB_001029c5,Dm9#2 (FAFB:11447061),distal medullary amacrine neuron Dm8,distal medullary amacrine neuron Dm9,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
112,VFB_001029ah,Dm8#11 (FAFB:10196189),2,VFB_001029b7,Dm#9 (FAFB:10537970),distal medullary amacrine neuron Dm8,distal medullary amacrine neuron,None,None,None,None
113,VFB_001029cl,Dm11#1 (FAFB:11450453),2,VFB_001029b7,Dm#9 (FAFB:10537970),distal medullary amacrine neuron Dm11,distal medullary amacrine neuron,None,None,None,None
114,VFB_001029ce,Dm2#1 (FAFB:11448822),2,VFB_001029b7,Dm#9 (FAFB:10537970),distal medullary amacrine neuron Dm2,distal medullary amacrine neuron,None,None,None,None
115,VFB_001029ae,Dm8#1 (FAFB:10109586),11,VFB_001029cg,Dm#2 (FAFB:11448962),distal medullary amacrine neuron Dm8,distal medullary amacrine neuron,None,None,None,None


In [39]:
# Consider adding query that => detailed MF profile of each Gene returned?

q = vc.nc.commit_list(["MATCH (c:Class:Neuron)<-[:composed_primarily_of]-(:Individual)-[:expresses]->(g:Gene:Glutamate_receptor)-[]->(go:Class) where go.curie =~ 'GO:.*' return distinct c.label as neuron, g.label as gene, collect(distinct go.label) as molecular_functions order by c.label"])


In [40]:
from vfb_connect.neo.neo4j_tools import dict_cursor
import pandas as pd
pd.DataFrame.from_records(dict_cursor(q))

,neuron,gene,molecular_functions
0,CCHamide-1 neuron,Nmdar2,"[NMDA glutamate receptor activity, transmitter..."
1,CCHamide-1 neuron,KaiR1D,[transmitter-gated monoatomic ion channel acti...
2,CCHamide-1 neuron,GluClα,"[Group 3 - Selective Cl- transport, neurotrans..."
3,CCHamide-1 neuron,Nmdar1,"[NMDA glutamate receptor activity, signaling r..."
4,CCHamide-1 neuron,CG11155,[Activation of highly calcium permeable postsy...
...,...,...,...
702,transmedullary neuron Tm9,GluClα,"[Group 3 - Selective Cl- transport, neurotrans..."
703,transmedullary neuron Tm9,CG11155,[Activation of highly calcium permeable postsy...
704,transmedullary neuron Tm9,GluRIB,[transmitter-gated monoatomic ion channel acti...
705,transmedullary neuron Tm9,GluRIA,[transmitter-gated monoatomic ion channel acti...


In [ ]:
# Consider adding queries to => broader GO annotation profiles for receptors ==> broader biological story.